In [2]:
import pandas as pd


In [3]:
df = pd.read_csv("rerank_dataset.csv", index_col=0)

df.tail(3)

,questions,answer,doc_page,doc_content,doc_source
90,ต้องทำอย่างไรเมื่อความเร็วรถลดลงต่ำกว่าประมาณ ...,"เมื่อความเร็วรถลดลงต่ำกว่าประมาณ 30 กม./ชม., ค...",127,แผงหน้าปัดและอุปกรณ์ควบคุม 5 \nคันสวิตช์ไฟเล...,docs/Text_PDF_.pdf
91,ระบบ FCM อาจไม่ทำงานในสภาวะใดบ้าง?,ระบบ FCM อาจไม่ทำงานในสภาวะดังต่อไปนี้:\n- รถค...,213,○ ระบบ FCM อาจไม่ทํางานหากรถคันหน้ามีลักษณะแคบ...,docs/Text_PDF_.pdf
92,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อแสดงสถานะการเปลี...,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อจะสว่างตามตำแหน่...,180,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อและตัวแสดงการทํ...,docs/Text_PDF_.pdf


In [5]:
df["doc_content"][0]

'และดึงสวิตช์ขึ้นเพื่อปิดกุญแจอัจฉริยะ หมายเหตุ -  การเปิดปิดซํ้าๆ ขณะที่เครื่องยนต์ไม่ทํางานจะเป็นการสิ้นเปลืองแบตเตอรี่พยายามอย่าใช้สวิตช์กระจกยกเว้นในขณะที่เครื่องยนต์กําลังทํางาน -  กระจกหน้าต่างประตูหลังจะเปิดเพียงครึ่งเดียวเท่านั้น (ดับเบิลแค็บ) สวิตช์ล็อก เมื่อกดสวิตช์ล็อก (1) ไฟแสดง (2) จะสว่างขึ้น และสวิตช์ด้านผู้โดยสารจะไม่สามารถใช้เปิดหรือปิดกระจกหน้าต่างประตูได้ และสวิตช์ด้านคนขับจะไม่สามารถเปิดหรือปิดกระจกหน้าต่างประตูอื่นได้นอกจากกระจกหน้าต่างประตูด้านคนขับการปลดล็อก ให้กดสวิตช์ล็อกอีกครั้ง และไฟแสดงจะดับลงกุญแจอัจฉริยะ'